In [3]:
# Example for linear regression:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class CustomLinear(Layer):
    def __init__(self, units):
        super(CustomLinear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
           shape = (input_shape[-1], self.units),
           initializer = "random_normal",
           trainable = True
        )
        self.b = self.add_weight(
           shape = (self.units,), 
           initializer = "random_normal",
           trainable = True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


In [21]:
# Once the layer is defined, the instance call will automatically initialize the weights of the layer:

# Definition of a CustomLinear layer with five neurons
linear = CustomLinear(5)
# Initialization of the weights and transformation of the data input
data = tf.ones((6, 1))
linear(data)

<tf.Tensor: shape=(6, 5), dtype=float32, numpy=
array([[ 0.04527579, -0.10158293, -0.1408679 , -0.01745914,  0.12282155],
       [ 0.04527579, -0.10158293, -0.1408679 , -0.01745914,  0.12282155],
       [ 0.04527579, -0.10158293, -0.1408679 , -0.01745914,  0.12282155],
       [ 0.04527579, -0.10158293, -0.1408679 , -0.01745914,  0.12282155],
       [ 0.04527579, -0.10158293, -0.1408679 , -0.01745914,  0.12282155],
       [ 0.04527579, -0.10158293, -0.1408679 , -0.01745914,  0.12282155]],
      dtype=float32)>

In [17]:
print("number of weights: ", len(linear.weights))
print("number of trainable weights: ", len(linear.trainable_weights))
print("number of non trainable weights",len(linear.non_trainable_weights))

number of weights:  2
number of trainable weights:  2
number of non trainable weights 0


In [4]:


from tensorflow.keras.layers import Layer


class CustomPolynomial(Layer):
    def __init__(self, units):
        super(CustomPolynomial, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1]*2, self.units),
            initializer="random_normal",
            trainable=True
        )
        self.b = self.add_weight(
            shape=(self.units,),
            initializer="random_normal",
            trainable=True
        )
# create a layer representing a second order polynomial regression, containing weights w and a bias b. 
# This layer will transform the inputs by applying a linear combination with the weights.
    def call(self, inputs):
        inputs_pow = tf.pow(inputs, 2)
        inputs_process = tf.concat([inputs, inputs_pow], axis=-1)
        return tf.matmul(inputs_process, self.w) + self.b

In [18]:
customlayer = CustomPolynomial(8)

# Definition of an input vector
inputs = tf.ones((4, 5))

# Apply the custom layer operation.
output = customlayer(inputs)
print(output)

tf.Tensor(
[[-0.29178956  0.09827712 -0.00870701 -0.20321968 -0.01325365  0.21515223
  -0.05782273 -0.00833619]
 [-0.29178956  0.09827712 -0.00870701 -0.20321968 -0.01325365  0.21515223
  -0.05782273 -0.00833619]
 [-0.29178956  0.09827712 -0.00870701 -0.20321968 -0.01325365  0.21515223
  -0.05782273 -0.00833619]
 [-0.29178956  0.09827712 -0.00870701 -0.20321968 -0.01325365  0.21515223
  -0.05782273 -0.00833619]], shape=(4, 8), dtype=float32)


In [19]:
print("number of weights: ", len(customlayer.weights))
print("number of trainable weights: ", len(customlayer.trainable_weights))
print("number of non trainable weights",len(customlayer.non_trainable_weights))

number of weights:  2
number of trainable weights:  2
number of non trainable weights 0


In [22]:
class CustomDropout(Layer):
    def __init__(self, rate):
        super(CustomDropout, self).__init__()
        self.rate = rate

    def call(self, inputs, training=None):
        if training:
            return tf.nn.dropout(inputs, rate=self.rate)
        return inputs

In [23]:
# custom model inherited from the Model class.

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Softmax

class CustomModel(Model):
    def __init__(self, units_1, units_2, n_classes):
        # Initialization of the Model class.
        super(CustomModel, self).__init__()
        # Definition of each layer of our model.
        self.linear_1 = CustomLinear(units_1)
        self.dropout_1 = CustomDropout(rate=0.2)

        self.linear_2 = CustomLinear(units_2)
        self.dropout_2 = CustomDropout(rate=0.2)

        self.linear_3 = CustomLinear(n_classes)
        self.softmax = Softmax()

    def call(self, inputs):
        # Apply the operation of each layer.
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.dropout_1(x)

        x = self.linear_2(x)
        x = tf.nn.relu(x)
        x = self.dropout_2(x)

        x = self.linear_3(x)
        return self.softmax(x)

In [24]:
from tensorflow.keras.layers import ReLU, Softmax

model = tf.keras.Sequential()

model.add(CustomPolynomial(32))
model.add(CustomDropout(0.2))
model.add(ReLU())

model.add(CustomPolynomial(64))
model.add(CustomDropout(0.2))
model.add(ReLU())

model.add(CustomPolynomial(1))
model.add(ReLU())

In [25]:
model(tf.ones([10, 5]))

<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[0.00670942],
       [0.00670942],
       [0.00670942],
       [0.00670942],
       [0.00670942],
       [0.00670942],
       [0.00670942],
       [0.00670942],
       [0.00670942],
       [0.00670942]], dtype=float32)>

In [27]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Set a random seed for reproducibility
np.random.seed(42)

# Create a dataset with 1000 samples and 5 feature columns
n_samples = 1000
n_features = 5

# Generate random feature data (values between 0 and 1)
features_data = np.random.rand(n_samples, n_features)

# Generate random target data (PressureLevel) - using values between 0 and 100
target_data = np.random.randint(0, 100, size=n_samples)

# Convert to a DataFrame for features and a Series for the target
columns = [f'feature_{i}' for i in range(1, n_features+1)]
df = pd.DataFrame(features_data, columns=columns)
df['PressureLevel'] = target_data

# Show the first few rows of the DataFrame
df.head()




,feature_1,feature_2,feature_3,feature_4,feature_5,PressureLevel
0,0.374540,0.950714,0.731994,0.598658,0.156019,50
1,0.155995,0.058084,0.866176,0.601115,0.708073,70
2,0.020584,0.969910,0.832443,0.212339,0.181825,95
3,0.183405,0.304242,0.524756,0.431945,0.291229,47
4,0.611853,0.139494,0.292145,0.366362,0.456070,18


In [28]:
# Split the dataset into features and target
features = df.drop(['PressureLevel'], axis=1)
target = df['PressureLevel']

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Print the shapes of the resulting datasets
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (800, 5)
X_test shape: (200, 5)
y_train shape: (800,)
y_test shape: (200,)


In [29]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)

y_train = y_train.ravel().reshape(-1, 1)
y_test = y_test.ravel().reshape(-1, 1)
y_scaler = StandardScaler().fit(y_train)

# Features
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

# Target
y_train = y_scaler.transform(y_train)
y_test = y_scaler.transform(y_test)

C:\Users\aryan\AppData\Local\Temp\ipykernel_3612\3483093910.py:5: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_train = y_train.ravel().reshape(-1, 1)
C:\Users\aryan\AppData\Local\Temp\ipykernel_3612\3483093910.py:6: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_test = y_test.ravel().reshape(-1, 1)


In [30]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=[tf.keras.metrics.MeanAbsoluteError()])

In [31]:
history = model.fit(X_train, y_train, 
                    batch_size=32,
                    epochs=100, 
                    validation_data=(X_test, y_test))


Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 1.0314 - mean_absolute_error: 0.8816 - val_loss: 0.9687 - val_mean_absolute_error: 0.8699
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9470 - mean_absolute_error: 0.8275 - val_loss: 0.9687 - val_mean_absolute_error: 0.8699
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0301 - mean_absolute_error: 0.8730 - val_loss: 0.9687 - val_mean_absolute_error: 0.8699
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9621 - mean_absolute_error: 0.8466 - val_loss: 0.9687 - val_mean_absolute_error: 0.8699
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0084 - mean_absolute_error: 0.8605 - val_loss: 0.9687 - val_mean_absolute_error: 0.8699
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0553 - mean_absolute_error: 0.8908 - val_loss: 0.9687 - val_mean_absolute_error: 0.8699
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9814 - mean_absolute_error: 0.8476 - val_loss